In [1]:
# Note: The model and training settings do not follow the reference settings
# from the paper. The settings are chosen such that the example can easily be
# run on a small dataset with a single GPU.

import pytorch_lightning as pl
import torch
import torchvision
from torchvision.transforms import Compose, Lambda
from torch import nn

from lightly.loss import NTXentLoss
from lightly.models.modules import SimCLRProjectionHead
from lightly.transforms.simclr_transform import SimCLRTransform

In [2]:
class SimCLR(pl.LightningModule):
    def __init__(self):
        super().__init__()
        resnet = torchvision.models.resnet18()
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])
        self.projection_head = SimCLRProjectionHead(512, 2048, 2048)
        self.criterion = NTXentLoss()

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(x)
        return z

    def training_step(self, batch, batch_index):
        (x0, x1) = batch[0]
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(self.parameters(), lr=0.06)
        return optim

In [3]:
model = SimCLR()

# this part is little tricky
# random_gray_scale=1.0
# but may work
# FIXME: start here next time
# FIXME: cleanup
# FIXME: prepare 2-3 clear notebooks
simclr_transform = SimCLRTransform(input_size=32, random_gray_scale=1.0)
mnist_to_3_channels = Lambda(lambda x: x.expand(3, -1, -1))
transform = Compose([mnist_to_3_channels, simclr_transform])

# dataset = torchvision.datasets.CIFAR10(
#     "datasets/cifar10", download=True, transform=transform, train=True
# )

# ... or MNIST
dataset = torchvision.datasets.MNIST(
    "datasets/mnist-clean-torchvision", download=True, transform=transform, train=True
)

# ... or create a dataset from a folder containing images or videos:
# dataset = LightlyDataset("path/to/folder", transform=transform)

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=256,
    shuffle=True,
    drop_last=True,
    num_workers=8,
)

In [4]:
accelerator = "gpu" if torch.cuda.is_available() else "cpu"
trainer = pl.Trainer(max_epochs=10, devices=1, accelerator=accelerator)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
d:\__repos\ml_concepts\venv\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [5]:
# training
trainer.fit(model=model, train_dataloaders=dataloader)

# read from checkpoint
# model = SimCLR.load_from_checkpoint(
#     r"lightning_logs\ "
#     )

You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                 | Params
---------------------------------------------------------
0 | backbone        | Sequential           | 11.2 M
1 | projection_head | SimCLRProjectionHead | 5.3 M 
2 | criterion       | NTXentLoss           | 0     
---------------------------------------------------------
16.4 M    Trainable params
0         Non-trainable params
16.4 M    Total params
65.710    Total estimated model params size (MB)
d:\__repos\ml_concepts\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_work

RuntimeError: Please call `iter(combined_loader)` first.

## Generate embeddings

In [ ]:
import matplotlib.pyplot as plt
from utils import generate_embeddings
from lightly.transforms.utils import IMAGENET_NORMALIZE

In [ ]:
test_transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=IMAGENET_NORMALIZE["mean"],
            std=IMAGENET_NORMALIZE["std"],
        ),
    ]
)

In [ ]:
# note, that this is not 'test' in a train/test sense
# SIMCLR is uses SSL method, so labels doesn't really matter
# test_dataset = torchvision.datasets.CIFAR10(
#     "datasets/test-cifar10", download=True, transform=test_transform, train=False
# )

test_dataset = torchvision.datasets.CIFAR10(
    "datasets/mnist-clean-torchvision",
    download=True,
    transform=test_transform,
    train=False,
)

In [ ]:
len(dataset)

In [ ]:
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=256,
    # this is important!
    shuffle=False,
    drop_last=True,
    num_workers=8,
)

In [ ]:
embeddings = generate_embeddings(model, test_dataloader)

In [ ]:
print(type(embeddings))
print(len(embeddings))
print(embeddings.shape)

### Random

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
num_principal_components = 3
n_clusters = 20

In [ ]:
pca = PCA(n_components=num_principal_components)
embeddings_reduced = pca.fit_transform(embeddings.to("cpu").numpy())

kmeans = KMeans(n_clusters=n_clusters)
labels = kmeans.fit_predict(embeddings_reduced)
centroids = kmeans.cluster_centers_

In [ ]:
centroids.shape

In [ ]:
ax = plt.figure().add_subplot(projection="3d")

ax.scatter(
    embeddings_reduced[:, 0],
    embeddings_reduced[:, 1],
    embeddings_reduced[:, 2],
    c=labels,
    alpha=0.05,
)

ax.scatter(
    centroids[:, 0],
    centroids[:, 1],
    centroids[:, 2],
    c="red",
    s=100,
    alpha=1.0,
)

plt.show()